# Grad-CAM

### 参数说明

1. sample_dir： 你自己的样本目录
2. model_root：你自己的模型目录，注意这里不需要精确到viz目录，所有的Grad-CAM图会生成的此目录的Grad-CAM文件夹
3. target_layer：你自己喜欢的层的名称，如果不知道具体的参数名字，可以先运行一次，在输入修改。

In [1]:
import os
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.comp2 import target_layer_mapping

sample_dir = get_param_in_cwd('data_pattern')
model_root = os.path.join(get_param_in_cwd('model_root'), get_param_in_cwd('model_name'))
target_layer = target_layer_mapping[get_param_in_cwd('model_name') + '_2D']

TypeError: join() argument must be str or bytes, not 'list'

In [ ]:
from onekey_algo.datasets.image_loader import default_loader
from onekey_algo.custom.components.comp2 import show_cam_on_image
import torch
import os
import random
from onekey_algo import get_param_in_cwd
from onekey_algo.custom.components.comp2 import extract, init_from_model, init_from_onekey
from onekey_algo.utils.MultiProcess import MultiProcess
import numpy as np
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import monai
from glob import glob
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib
matplotlib.use('Agg')

samples = glob(os.path.join(sample_dir, '*'))
random.shuffle(samples)
def viz_sample(samples, thread_id):
    model, transformer, device = init_from_onekey(os.path.join(model_root, 'viz'))
    for n, m in model.named_modules():
        print('Feature name:', n, "|| Module:", m)
#     target_layer = "layer4.1.conv2"
#     target_layer = 'features.denseblock4.denselayer32.conv2'
#     target_layer = 'layers.3.1.layers.1.3.4'
#     target_layer = 'avgpool'
#     return 
    gradcam = monai.visualize.GradCAM(nn_module=model, target_layers=target_layer)
#     return
    viz_dir = os.path.join(model_root, 'Grad-CAM')
    os.makedirs(viz_dir, exist_ok=True)
    for sample in samples:
        if os.path.exists(os.path.join(viz_dir, os.path.basename(sample))):
            continue
        img = default_loader(sample)
        sample_ = transformer(img)
        sample_  = sample_.view(1, *sample_.size()).to(device)
        res_cam = gradcam(x=sample_, class_idx=None)
        fig, axes = plt.subplots(1, 3, figsize=(12, 4), facecolor='white')
    #     axes[0].imshow(-res_cam[0][0].cpu(), cmap='jet')
        axes[0].imshow(img.resize(sample_.size()[2:]))
        axes[0].axis('off')
    #     plt.savefig(f"viz/{os.path.basename(sample).replace('.png', '_se.png')}", bbox_inches = 'tight')
    #     plt.show()
    #     plt.figure(figsize=(10, 10))
    #     plt.axis('off')
        imshow = axes[1].imshow(-res_cam[0][0].cpu(),cmap='jet')
        axes[1].axis('off')
        imshow = axes[2].imshow(show_cam_on_image(img.resize(sample_.size()[2:]), -res_cam[0][0].cpu(), use_rgb=True, reverse=False), 
                                cmap='jet')
        axes[2].axis('off')
        cax = fig.add_axes([0.92, 0.17, 0.02, axes[2].get_position().height]) 
        plt.colorbar(imshow, cax=cax)
        plt.savefig(f'{viz_dir}/{os.path.basename(sample).replace(".npy", ".png")}', bbox_inches = 'tight')
        plt.show()
        plt.close(fig)
viz_sample(samples, thread_id=1)
# MultiProcess(func=viz_sample, samples=samples, num_process=1).run()